In [1]:
import os

model_root = '../results/models'
files = os.listdir(model_root)
params = ['_'.join(f.split('_')[2:]) for f in files]
params = [p.split('.pkl')[0] for p in params]
params = list(set(params))
params

['p_19.0_q_9.122152261131532_g_1',
 'p_19.0_q_8.483911078685804_g_2',
 'p_0.5_q_1.895944090041435_g_1',
 'p_5.5_q_9.010757520712524_g_1',
 'p_7.305688086564288_q_7.517332462471247_g_2']

In [2]:
import pandas as pd
import pickle

for p in params:
    emb_fp = f'../data/emb/from_adelle/emb_{p}.tsv'
    emb = pd.read_csv(emb_fp, sep='\t', index_col=0)
    samples = emb[emb.index.str.startswith('MD')]
    end_samples = samples[samples.index.str.contains('End')]
    features = emb[~emb.index.str.startswith('MD')]
    for classifier_type, colnames in zip(['time', 'diet'], [['baseline', 'endpoint'], ['dairy', 'meat']]):
        for model_num in range(1,6):
            model_fp = f'{model_root}/{classifier_type}_{model_num}_{p}.pkl'
            if not os.path.exists(model_fp):
                continue
            with open(model_fp, 'rb') as f:
                model = pickle.load(f)
            feature_preds = model.predict_proba(features)
            feature_preds = pd.DataFrame(feature_preds, index=features.index, columns=colnames)
            if classifier_type == 'time':
                sample_preds = model.predict_proba(samples)
                sample_preds = pd.DataFrame(sample_preds, index=samples.index, columns=colnames)

            else:
                sample_preds = model.predict_proba(end_samples)
                sample_preds = pd.DataFrame(sample_preds, index=end_samples.index, columns=colnames)

            feature_preds.to_csv(f'/mnt/home/f0106093/Projects/multiomics-embedding/results/predictions/{classifier_type}_{model_num}_{p}_features.tsv', sep='\t')
            sample_preds.to_csv(f'/mnt/home/f0106093/Projects/multiomics-embedding/results/predictions/{classifier_type}_{model_num}_{p}_samples.tsv', sep='\t')

/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegre